In [1]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
# Загружаем датасет Iris
data = load_iris()
X, y = data.data, data.target

# Нормализация данных
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Преобразуем данные в тензоры PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

In [11]:
y_test

tensor([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2,
        2, 2, 2, 2, 0, 0])

In [3]:
# Определение квантовой схемы
dev = qml.device("default.qubit", wires=4)

def quantum_layer(inputs):
    for i in range(4):
        qml.RY(inputs[i], wires=i)
    for i in range(3):
        qml.CNOT(wires=[i, i+1])
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

qnode = qml.QNode(quantum_layer, dev, interface='torch')

# Определение квантового слоя
class QuantumLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return torch.stack([torch.tensor(qnode(x_i), dtype=torch.float32) for x_i in x])

# Определение гибридной квантово-классической LSTM модели
class HybridQuantumLSTM(nn.Module):
    def __init__(self):
        super(HybridQuantumLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=4, hidden_size=8, num_layers=1, batch_first=True)
        self.quantum_layer = QuantumLayer()
        self.fc = nn.Linear(4, 3)

    def forward(self, x):
        lstm_out, _ = self.lstm(x.unsqueeze(1))
        quantum_out = self.quantum_layer(lstm_out.squeeze(1))
        return self.fc(quantum_out)

In [4]:
# Инициализация модели, функции потерь и оптимизатора
model = HybridQuantumLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [8]:
# Обучение модели
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 1.0993
Epoch [2/100], Loss: 1.0989
Epoch [3/100], Loss: 1.0987
Epoch [4/100], Loss: 1.0987
Epoch [5/100], Loss: 1.0987
Epoch [6/100], Loss: 1.0989
Epoch [7/100], Loss: 1.0990
Epoch [8/100], Loss: 1.0991
Epoch [9/100], Loss: 1.0992
Epoch [10/100], Loss: 1.0991
Epoch [11/100], Loss: 1.0989
Epoch [12/100], Loss: 1.0987
Epoch [13/100], Loss: 1.0985
Epoch [14/100], Loss: 1.0982
Epoch [15/100], Loss: 1.0980
Epoch [16/100], Loss: 1.0978
Epoch [17/100], Loss: 1.0977
Epoch [18/100], Loss: 1.0976
Epoch [19/100], Loss: 1.0975
Epoch [20/100], Loss: 1.0975
Epoch [21/100], Loss: 1.0974
Epoch [22/100], Loss: 1.0974
Epoch [23/100], Loss: 1.0974
Epoch [24/100], Loss: 1.0973
Epoch [25/100], Loss: 1.0972
Epoch [26/100], Loss: 1.0971
Epoch [27/100], Loss: 1.0969
Epoch [28/100], Loss: 1.0968
Epoch [29/100], Loss: 1.0967
Epoch [30/100], Loss: 1.0966
Epoch [31/100], Loss: 1.0965
Epoch [32/100], Loss: 1.0964
Epoch [33/100], Loss: 1.0963
Epoch [34/100], Loss: 1.0962
Epoch [35/100], Loss: 1

In [10]:
# Тестирование модели
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_classes = torch.argmax(y_pred, dim=1)
    accuracy = (y_pred_classes == y_test).float().mean()
    print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.3000
